## Info

All codes you can find here: https://github.com/CloseToAlgoTrading/Kaggle_GFootball

In the repo you can find a Dockerfile, so you can start gfootball environment localy with Tensorflow and TF-agents

I used here a simle DQN, but we can change it to another agent. Check points and policy will be stored, so we can continue the traning, and we can use the policy for the submission. 

P.s. I'm not an expert in the reinforcement learning and my python code is also not optimal :), but I hope this notebook can help to someone understand the main steps. 
Oh.. if you find any issues with this implementation, please not be silent, because i'm learning too. Thanks.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install dependencies

In [ ]:
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
! pip install tensorflow
! pip install tf-agents


# Human agent

Human agent will be used for data collection

In [ ]:
from enum import Enum

class Action(Enum):
    Idle = 0
    Left = 1
    TopLeft = 2
    Top = 3
    TopRight = 4
    Right = 5
    BottomRight = 6
    Bottom = 7
    BottomLeft = 8
    LongPass= 9
    HighPass = 10
    ShortPass = 11
    Shot = 12
    Sprint = 13
    ReleaseDirection = 14
    ReleaseSprint = 15
    Slide = 16
    Dribble = 17
    ReleaseDribble = 18
    NotUsed = 55


sticky_index_to_action = [
    Action.Left,
    Action.TopLeft,
    Action.Top,
    Action.TopRight,
    Action.Right,
    Action.BottomRight,
    Action.Bottom,
    Action.BottomLeft,
    Action.Sprint,
    Action.Dribble
]


class PlayerRole(Enum):
    GoalKeeper = 0
    CenterBack = 1
    LeftBack = 2
    RightBack = 3
    DefenceMidfield = 4
    CentralMidfield = 5
    LeftMidfield = 6
    RIghtMidfield = 7
    AttackMidfield = 8
    CentralFront = 9


class GameMode(Enum):
    Normal = 0
    KickOff = 1
    GoalKick = 2
    FreeKick = 3
    Corner = 4
    ThrowIn = 5
    Penalty = 6

def simple_human_policy(obs):
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return Action.Shot
        # Run towards the goal otherwise.
        return Action.Right
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide
    

def human_agent_wrapper(obs):
    # Extract observations for the first (and only) player we control.
    obs_hr = obs['players_raw'][0]
    # Turn 'sticky_actions' into a set of active actions (strongly typed).
    obs_hr['sticky_actions'] = { sticky_index_to_action[nr] for nr, action in enumerate(obs_hr['sticky_actions']) if action }
    # Turn 'game_mode' into an enum.
    obs_hr['game_mode'] = GameMode(obs_hr['game_mode'])
    return obs_hr


#def agent(obs):

#    a =  simple_human_policy(human_agent_wrapper(obs)).value
    
#    return [a]



# TF-agent policy

I'm not expert on the user defined policy, but here I just create a simple policy that will use our agent for generate actions.

In [ ]:
"""Policy implementation that generates random actions."""
from __future__ import absolute_import
from __future__ import division
# Using Type Annotations.
from __future__ import print_function

from typing import Optional, Sequence

import numpy as np
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
from tf_agents.distributions import masked
from tf_agents.policies import py_policy
from tf_agents.specs import array_spec
from tf_agents.trajectories import policy_step
from tf_agents.trajectories import time_step as ts
from tf_agents.typing import types
from tf_agents.utils import nest_utils
from tf_agents.environments import utils


class MyFootbalHumanPyPolicy(py_policy.PyPolicy):
    """Returns random samples of the given action_spec."""
    
    
    def __convert_tf_obs_to_numpy(self, obs):
        
        rp = {}
        new_obs = {}
        for name in obs.keys():
            rp[name] = obs[name].numpy()[0] if len(obs[name].numpy()[0]) > 1 else obs[name].numpy()[0][0]

        new_obs['players_raw'] = [rp]
        return new_obs

    
    def __init__(self,
               time_step_spec: ts.TimeStep,
               action_spec: types.NestedArraySpec,
               info_spec: types.NestedArraySpec = (),
               seed: Optional[types.Seed] = None,
               outer_dims: Optional[Sequence[int]] = None,
               observation_and_action_constraint_splitter: Optional[
                   types.Splitter] = None):
        self._seed = seed
        self._outer_dims = outer_dims
        
        self._rng = np.random.RandomState(seed)
        if time_step_spec is None:
            time_step_spec = ts.time_step_spec()

        super(MyFootbalHumanPyPolicy, self).__init__(
            time_step_spec=time_step_spec,
            action_spec=action_spec,
            info_spec=info_spec,
            observation_and_action_constraint_splitter=(
                observation_and_action_constraint_splitter))

    def _action(self, time_step, policy_state):
        
        outer_dims = self._outer_dims
        
        if outer_dims is None:
            if self.time_step_spec.observation:
                outer_dims = nest_utils.get_outer_array_shape(
                    time_step.observation, self.time_step_spec.observation)
            else:
                outer_dims = ()

        random_action = np.array([simple_human_policy(human_agent_wrapper(self.__convert_tf_obs_to_numpy(time_step.observation))).value])

        info = array_spec.sample_spec_nest(
            self._info_spec, self._rng, outer_dims=outer_dims)

        return policy_step.PolicyStep(random_action, policy_state, info)



# Create Environment
I wrapped gfootball environmet. So we can tweak a rewards, if we wnat.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.utils import common

import gfootball
import gfootball.env as football_env

data_dic = {
    'ball_owned_team':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=2, name='ball_owned_team'),
    'steps_left':array_spec.BoundedArraySpec(shape=(1,), dtype=np.float32, minimum=0.0, name='steps_left'),
    'ball_owned_player':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=11, name='ball_owned_player'),
    'game_mode':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=6, name='game_mode'),
    'designated':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=-1, maximum=11, name='designated'),
    'active':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=-1, maximum=11, name='active'),
    'left_team_active':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='left_team_active'),
    'left_team':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='left_team'),
    'right_team_active':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='right_team_active'),
    'ball_direction':array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball_direction'),
    'ball': array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball'),
    'left_team_tired_factor':array_spec.BoundedArraySpec(shape=(11,), dtype=np.float32, minimum=0.0, name='left_team_tired_factor'),
    'left_team_direction':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='left_team_direction'),
    'score':array_spec.BoundedArraySpec(shape=(2,), dtype=np.float32, minimum=0.0, name='score'),
    'left_team_roles':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=10, name='left_team_roles'),
    'right_team_tired_factor':array_spec.BoundedArraySpec(shape=(11,), dtype=np.float32, minimum=0.0, name='right_team_tired_factor'),
    'right_team':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='right_team'),
    'right_team_direction':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='right_team_direction'),
    'right_team_yellow_card':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='right_team_yellow_card'),
    'right_team_roles':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=10, name='right_team_roles'),
    'ball_rotation':array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball_rotation'),
    'left_team_yellow_card':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='left_team_yellow_card'),
    'sticky_actions':array_spec.BoundedArraySpec(shape=(10,), dtype=np.int32, minimum=0, maximum=1, name='sticky_actions'),
    }

class MyFootballEnv(py_environment.PyEnvironment):

    @staticmethod
    def convert_observation_to_tf(obs, data_cfg, propertie_list = []):
        
        if(len(propertie_list) == 0):
            propertie_list = ['right_team_direction', 'right_team_tired_factor', 
                               'left_team_roles', 'left_team_direction', 'ball_direction', 
                               'ball_owned_player', 'right_team_yellow_card', 'ball', 
                               'right_team', 'steps_left', 'ball_rotation', 
                               'ball_owned_team', 'game_mode', 'left_team_yellow_card', 
                               'left_team', 'right_team_roles', 'right_team_active', 
                               'left_team_active', 'left_team_tired_factor', 'score', 
                               'designated', 'active', 'sticky_actions']
        rp = {}

        for name in propertie_list:
            
            if(('ball_owned_player' == name) and (obs[0][name] == -1)):
                obs[0][name] = 11
            if(('ball_owned_team' == name) and (obs[0][name] == -1)):
                obs[0][name] = 2
            
            if('right_team_roles' == name):
                if((obs[0]['ball_owned_team'] == 1) and ((obs[0]['ball_owned_player'] != -1) or (obs[0]['ball_owned_player'] != 11))):
                    obs[0][name] = obs[0][name][obs[0]['ball_owned_player']]
                else:
                    obs[0][name] = 10

            if('left_team_roles' == name):
                if((obs[0]['ball_owned_team'] == 0) and ((obs[0]['ball_owned_player'] != -1) or (obs[0]['ball_owned_player'] != 11))):
                    obs[0][name] = obs[0][name][obs[0]['ball_owned_player']]
                else:
                    obs[0][name] = 10


            if(data_cfg[name].shape == (1,)):
                rp[name] = np.array([np.squeeze(obs[0][name])]).astype(data_cfg[name].dtype)
            else:
                a = np.zeros(data_cfg[name].shape, dtype=data_cfg[name].dtype)
                a[:len(obs[0][name])] = obs[0][name]
                obs[0][name] = a.copy()
                rp[name] = np.array(obs[0][name]).astype(data_cfg[name].dtype)

        return rp

    @staticmethod
    def construct_obs_spec(data_cfg, cfg):
        rp = {}
        
        for name in cfg:
            rp[name] = data_cfg[name]
        
        return rp
        
    
    def __init__(self, propertie_list = [], scenario='11_vs_11_kaggle'):
        super().__init__()
        self._data_dic = data_dic
        
        if(len(propertie_list) == 0):
            propertie_list = ['right_team_direction', 'right_team_tired_factor', 
                               'left_team_roles', 'left_team_direction', 'ball_direction', 
                               'ball_owned_player', 'right_team_yellow_card', 'ball', 
                               'right_team', 'steps_left', 'ball_rotation', 
                               'ball_owned_team', 'game_mode', 'left_team_yellow_card', 
                               'left_team', 'right_team_roles', 'right_team_active', 
                               'left_team_active', 'left_team_tired_factor', 'score', 
                               'designated', 'active', 'sticky_actions']

        self.propertie_list = propertie_list
        
        self.env = football_env.create_environment(
              env_name=scenario,  
              stacked=False,
              representation='raw',  
              rewards='scoring, checkpoints',
              write_goal_dumps=False,
              write_full_episode_dumps=False,
              render=False,
              write_video=False,
              dump_frequency=1,
              logdir='./',
              extra_players=None,
              number_of_left_players_agent_controls=1,
              number_of_right_players_agent_controls=0
            )
        
        
        self._state = self.convert_observation_to_tf(self.env.reset(), self._data_dic, self.propertie_list)

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=18, name='action')
        
        
        #representation of the enviroment: price + open position state
        self._observation_spec = self.construct_obs_spec(self._data_dic, self.propertie_list)

        #used for idndication of the end of episode        
        self._episode_ended = False

        pass
       
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._episode_ended = False
        self._state = self.convert_observation_to_tf(self.env.reset(), self._data_dic, self.propertie_list)
        return ts.restart(self._state)


    def _step(self, action):

        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()
        
        self._state, reward, self._episode_ended, info = self.env.step(action)
        self._state = self.convert_observation_to_tf(self._state, self._data_dic, self.propertie_list)

        if self._episode_ended:
            return ts.termination(self._state, reward)
        else:
            return ts.transition(self._state, reward=reward, discount=1.0)



# Create and train agent

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

from pathlib import Path
import os

from tf_agents.drivers import dynamic_episode_driver
from tf_agents.eval import metric_utils
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.policies import policy_saver
from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network

from tf_agents.metrics import tf_metrics

import gym
import gfootball

tf.compat.v1.enable_v2_behavior()
tf.random.set_seed(12)
tf.print(tf.config.list_physical_devices('GPU'))

Configuration of the input data

In [ ]:
cfg = ['ball_owned_team', 
       'steps_left', 
       'ball_owned_player', 
       'game_mode', 
       #'designated', 
       'active', 
       #'left_team_active', 
       'left_team', 
       #'right_team_active', 
       'ball_direction', 
       'ball', 
       #'left_team_tired_factor', 
       'left_team_direction', 
       'score', 
       #'left_team_roles', 
       #'right_team_tired_factor', 
       'right_team', 
       'right_team_direction', 
       #'right_team_yellow_card', 
       #'right_team_roles', 
       'ball_rotation', 
       #'left_team_yellow_card', 
       'sticky_actions']

In [ ]:
# create test and train environment
train_env = tf_py_environment.TFPyEnvironment(MyFootballEnv(cfg, scenario='1_vs_1_easy'))
tf_env_eval = tf_py_environment.TFPyEnvironment(MyFootballEnv(cfg, scenario='1_vs_1_easy'))

In [ ]:
# helper class for learning agent
import time
class learningHelper:
    def __init__(self, train_env, test_env, agent, global_step, chkpdir='./',
        num_iterations=20000, collect_episodes=100, collect_steps_per_iteration=2,
        replay_buffer_capacity=20000, batch_size=64, log_interval=500, 
        num_eval_episodes=10, eval_interval = 5000, IsAutoStoreCheckpoint = True, collect_policy = None):

        if collect_policy is None:
            self.collect_policy = self.agent.collect_policy
            print('selected agent collect_policy')
        else:
            self.collect_policy = collect_policy
            print('selected USER collect_policy')
        
        tf.compat.v1.enable_v2_behavior()
        
        self.IsAutoStoreCheckpoint = IsAutoStoreCheckpoint
        self.num_iterations = num_iterations
        self.collect_episodes = collect_episodes
        self.collect_steps_per_iteration = collect_steps_per_iteration
        self.replay_buffer_capacity = replay_buffer_capacity

        self.batch_size = batch_size
        self.log_interval = log_interval

        self.num_eval_episodes = num_eval_episodes
        self.eval_interval = eval_interval
        
        self.agent = agent

        self.train_env = train_env
        self.test_env = test_env

        self.global_step = global_step

        #create reply buffer for collection trajactories
        self.replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
            self.agent.collect_data_spec,
            batch_size=self.train_env.batch_size,
            max_length=self.replay_buffer_capacity)

        #Checkpointer
        self.checkpoint_dir = os.path.join(chkpdir, 'checkpoint')
        Path(self.checkpoint_dir).mkdir(parents=True, exist_ok=True)
        self.policy_dir = os.path.join(chkpdir, 'policy')
        Path(self.policy_dir).mkdir(parents=True, exist_ok=True)

        self.train_checkpointer = common.Checkpointer(
            ckpt_dir=self.checkpoint_dir,
            max_to_keep=1,
            agent=self.agent,
            policy=self.agent.policy,
            replay_buffer=self.replay_buffer,
            global_step=self.global_step
            )

        self.tf_policy_saver = policy_saver.PolicySaver(self.agent.policy)
        
        self.local_step_counter = 0

        pass

    def evaluate_agent(self, n_episodes=100):
        #define metrics
        num_episodes = tf_metrics.NumberOfEpisodes()
        env_steps = tf_metrics.EnvironmentSteps()
        average_return = tf_metrics.AverageReturnMetric()
        #rew = TFSumOfRewards()

        #add reply buffer and metrict to the observer
        observers = [num_episodes, env_steps, average_return ]

        _driver = dynamic_episode_driver.DynamicEpisodeDriver(self.test_env, self.agent.policy, observers, num_episodes=n_episodes)

        final_time_step, _ = _driver.run()

        print('eval episodes = {0}: Average Return = {1}'.format(num_episodes.result().numpy(), average_return.result().numpy()))
        return average_return.result().numpy()

    def collect_training_data(self, verbose=0):

        if(verbose > 0):
            #define metrics
            num_episodes = tf_metrics.NumberOfEpisodes()
            env_steps = tf_metrics.EnvironmentSteps()
            #add reply buffer and metrict to the observer
            observers = [self.replay_buffer.add_batch, num_episodes, env_steps]
        else:
            observers = [self.replay_buffer.add_batch]

        self.replay_buffer.clear()
        #create a driver
        #we can create a driver using e.g. random policy
        driver = dynamic_episode_driver.DynamicEpisodeDriver(
            self.train_env, self.collect_policy, observers, num_episodes=self.collect_episodes)

        #collect_steps_per_iteration = 2
        #driver = dynamic_step_driver.DynamicStepDriver(
        #    train_env, tf_policy, observers, num_steps=collect_steps_per_iteration)

        # Initial driver.run will reset the environment and initialize the policy.
        final_time_step, policy_state = driver.run()
        if(verbose > 0):
            #print('final_time_step', final_time_step)
            print('Number of Steps: ', env_steps.result().numpy())
            print('Number of Episodes: ', num_episodes.result().numpy())

        pass 

    def train_step(self, n_steps):
        # Convert the replay buffer to a tf.data.Dataset 
        # Dataset generates trajectories with shape [Bx2x...]
        AUTOTUNE = tf.data.experimental.AUTOTUNE
        dataset = self.replay_buffer.as_dataset(
            num_parallel_calls=AUTOTUNE, 
            sample_batch_size=self.batch_size, 
            num_steps=2).prefetch(AUTOTUNE)

        iterator = iter(dataset)

        train_loss = None
        #experience = self.replay_buffer.gather_all()
        #train_loss = self.agent.train(experience) 
        for _ in range(n_steps):
            # Sample a batch of data from the buffer and update the agent's network.
            experience, unused_info = next(iterator)
            train_loss = self.agent.train(experience)            
        
        print('Global steps {}: Traning Loss {}'.format(self.global_step.numpy(), train_loss.loss))

    def train_agent(self, n_epoch):
        local_epoch_counter = 0
        for i in range(n_epoch):
            start_time = time.time()
            self.collect_training_data(verbose=0)
            #print('num_frames()', self.replay_buffer.num_frames().numpy())
            #print('n_steps()', int(self.replay_buffer.num_frames().numpy()/self.batch_size))
            self.train_step(self.replay_buffer.num_frames().numpy())
            
            if(self.IsAutoStoreCheckpoint == True):
                self.store_check_point()
            epoch_train_time = time.time() - start_time
            local_epoch_counter = local_epoch_counter + 1
            print('Epoch: {}, epoch train time: {}'.format( local_epoch_counter, epoch_train_time ))
        pass

    def train_agent_with_avg_ret_condition(self, max_steps, min_avg_return, n_eval_steps=100):
        for i in range(max_steps):
            self.collect_training_data()
            self.train_step(1000)
            if(self.IsAutoStoreCheckpoint == True):
                self.store_check_point()
            
            if ((i>0) and (i % self.eval_interval) == 0):
                avg_ret = self.evaluate_agent(n_eval_steps)
                if(avg_ret > min_avg_return):
                    return
        pass

    def get_agent(self):
        return self.agent

    def store_check_point(self):
        self.train_checkpointer.save(self.global_step)
        pass
    def restore_check_point(self):
        self.train_checkpointer.initialize_or_restore()
        self.global_step = tf.compat.v1.train.get_global_step()
        pass
    def save_policy(self):
        self.tf_policy_saver.save(self.policy_dir)
        pass

# Create a Q_Network

In [ ]:
fc_layer_params = (100,50)

preprocessing_layers_d = {
    'left_team_yellow_card': tf.keras.layers.Dense(11),
    'left_team_roles': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=11),#tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=10),
                        #                           tf.keras.layers.Flatten()]),

    'ball_direction': tf.keras.layers.Dense(3),
    'left_team_tired_factor': tf.keras.layers.Dense(11),
    'left_team_active': tf.keras.layers.Dense(11),
    'right_team_tired_factor': tf.keras.layers.Dense(11),
    'ball': tf.keras.layers.Dense(3),
    'ball_owned_player': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12),#tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12),
                                                   #tf.keras.layers.Flatten()]),
    'ball_rotation': tf.keras.layers.Dense(1),
    'right_team_active': tf.keras.layers.Dense(11), 
    'game_mode': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=7),#tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=7),
                                                   #tf.keras.layers.Flatten()]),
    'steps_left': tf.keras.layers.Dense(1),
    'right_team': tf.keras.layers.Flatten(),
    'right_team_yellow_card': tf.keras.layers.Dense(11),
    'left_team': tf.keras.layers.Flatten(),
    'ball_owned_team': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=3),#tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=3),
                                                  #tf.keras.layers.Flatten()]),
    'score': tf.keras.layers.Dense(2),
    'right_team_roles': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=11), #tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12),
                                                   #tf.keras.layers.Flatten()]),
    'right_team_direction': tf.keras.layers.Flatten(),
    'left_team_direction': tf.keras.layers.Flatten(),
    'designated': tf.keras.layers.Dense(1),
    'active': tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12),#tf.keras.models.Sequential([tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12),
                                                   #tf.keras.layers.Flatten()]),
    'sticky_actions': tf.keras.layers.Dense(10),
    }

preprocessing_layers = {}
for k in cfg:
    preprocessing_layers[k] = preprocessing_layers_d[k]

preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    preprocessing_layers=preprocessing_layers,
    preprocessing_combiner=preprocessing_combiner,
    fc_layer_params=fc_layer_params)

# Create DQN agent

In [ ]:
learning_rate = 1e-3  

#create optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

#create a global step coubter
#train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

#create agent
agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=global_step,
    target_update_period = 10
)

agent.initialize()

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)


# Create policy for data collection

In [ ]:
a_spec = train_env.action_spec()
t_spec = train_env.time_step_spec()
mp = MyFootbalHumanPyPolicy(time_step_spec=t_spec, action_spec=a_spec)

# Agent learning object

In [ ]:
magent = learningHelper(train_env=train_env, test_env=tf_env_eval, agent=agent, global_step=global_step, 
                        collect_episodes = 3,
                        eval_interval=5,
                        replay_buffer_capacity=3500,
                        batch_size=500,
                        collect_policy = mp
)
magent.restore_check_point()

# Train one cycle
I'm using here rule based policy for data collection

In [ ]:
%%time
magent.train_agent(1)
magent.save_policy()

# Evaluate agent
Evaluation is done on one episode

In [ ]:
avg_ret = magent.evaluate_agent(1)
print(avg_ret)

# Create a submission file

As all code is in the kaggle notebook, I copied some code above into the submission file directly

In [ ]:
%%writefile submission.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from kaggle_environments.envs.football.helpers import *

import tensorflow as tf
import numpy as np
from tf_agents.trajectories import time_step as ts

import abc
import tensorflow as tf
import numpy as np
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.utils import common

import gfootball
import gfootball.env as football_env

data_dic = {
    'ball_owned_team':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=2, name='ball_owned_team'),
    'steps_left':array_spec.BoundedArraySpec(shape=(1,), dtype=np.float32, minimum=0.0, name='steps_left'),
    'ball_owned_player':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=11, name='ball_owned_player'),
    'game_mode':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=6, name='game_mode'),
    'designated':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=-1, maximum=11, name='designated'),
    'active':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=-1, maximum=11, name='active'),
    'left_team_active':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='left_team_active'),
    'left_team':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='left_team'),
    'right_team_active':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='right_team_active'),
    'ball_direction':array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball_direction'),
    'ball': array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball'),
    'left_team_tired_factor':array_spec.BoundedArraySpec(shape=(11,), dtype=np.float32, minimum=0.0, name='left_team_tired_factor'),
    'left_team_direction':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='left_team_direction'),
    'score':array_spec.BoundedArraySpec(shape=(2,), dtype=np.float32, minimum=0.0, name='score'),
    'left_team_roles':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=10, name='left_team_roles'),
    'right_team_tired_factor':array_spec.BoundedArraySpec(shape=(11,), dtype=np.float32, minimum=0.0, name='right_team_tired_factor'),
    'right_team':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='right_team'),
    'right_team_direction':array_spec.BoundedArraySpec(shape=(11,2), dtype=np.float32, minimum=-1.5, maximum=1.5, name='right_team_direction'),
    'right_team_yellow_card':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='right_team_yellow_card'),
    'right_team_roles':array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, maximum=10, name='right_team_roles'),
    'ball_rotation':array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='ball_rotation'),
    'left_team_yellow_card':array_spec.BoundedArraySpec(shape=(11,), dtype=np.int32, minimum=0, maximum=1, name='left_team_yellow_card'),
    'sticky_actions':array_spec.BoundedArraySpec(shape=(10,), dtype=np.int32, minimum=0, maximum=1, name='sticky_actions'),
    }

class MyFootballEnv(py_environment.PyEnvironment):

    @staticmethod
    def convert_observation_to_tf(obs, data_cfg, propertie_list = []):
        
        if(len(propertie_list) == 0):
            propertie_list = ['right_team_direction', 'right_team_tired_factor', 
                               'left_team_roles', 'left_team_direction', 'ball_direction', 
                               'ball_owned_player', 'right_team_yellow_card', 'ball', 
                               'right_team', 'steps_left', 'ball_rotation', 
                               'ball_owned_team', 'game_mode', 'left_team_yellow_card', 
                               'left_team', 'right_team_roles', 'right_team_active', 
                               'left_team_active', 'left_team_tired_factor', 'score', 
                               'designated', 'active', 'sticky_actions']
        rp = {}

        for name in propertie_list:
            
            if(('ball_owned_player' == name) and (obs[0][name] == -1)):
                obs[0][name] = 11
            if(('ball_owned_team' == name) and (obs[0][name] == -1)):
                obs[0][name] = 2
            
            if('right_team_roles' == name):
                if((obs[0]['ball_owned_team'] == 1) and ((obs[0]['ball_owned_player'] != -1) or (obs[0]['ball_owned_player'] != 11))):
                    obs[0][name] = obs[0][name][obs[0]['ball_owned_player']]
                else:
                    obs[0][name] = 10

            if('left_team_roles' == name):
                if((obs[0]['ball_owned_team'] == 0) and ((obs[0]['ball_owned_player'] != -1) or (obs[0]['ball_owned_player'] != 11))):
                    obs[0][name] = obs[0][name][obs[0]['ball_owned_player']]
                else:
                    obs[0][name] = 10


            if(data_cfg[name].shape == (1,)):
                rp[name] = np.array([np.squeeze(obs[0][name])]).astype(data_cfg[name].dtype)
            else:
                a = np.zeros(data_cfg[name].shape, dtype=data_cfg[name].dtype)
                a[:len(obs[0][name])] = obs[0][name]
                obs[0][name] = a.copy()
                rp[name] = np.array(obs[0][name]).astype(data_cfg[name].dtype)

        return rp

    @staticmethod
    def construct_obs_spec(data_cfg, cfg):
        rp = {}
        
        for name in cfg:
            rp[name] = data_cfg[name]
        
        return rp
        
    
    def __init__(self, propertie_list = [], scenario='11_vs_11_kaggle'):
        super().__init__()
        self._data_dic = data_dic
        
        if(len(propertie_list) == 0):
            propertie_list = ['right_team_direction', 'right_team_tired_factor', 
                               'left_team_roles', 'left_team_direction', 'ball_direction', 
                               'ball_owned_player', 'right_team_yellow_card', 'ball', 
                               'right_team', 'steps_left', 'ball_rotation', 
                               'ball_owned_team', 'game_mode', 'left_team_yellow_card', 
                               'left_team', 'right_team_roles', 'right_team_active', 
                               'left_team_active', 'left_team_tired_factor', 'score', 
                               'designated', 'active', 'sticky_actions']

        self.propertie_list = propertie_list
        
        self.env = football_env.create_environment(
              env_name=scenario,  
              stacked=False,
              representation='raw',  
              rewards='scoring, checkpoints',
              write_goal_dumps=False,
              write_full_episode_dumps=False,
              render=False,
              write_video=False,
              dump_frequency=1,
              logdir='./',
              extra_players=None,
              number_of_left_players_agent_controls=1,
              number_of_right_players_agent_controls=0
            )
        
        
        self._state = self.convert_observation_to_tf(self.env.reset(), self._data_dic, self.propertie_list)

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=18, name='action')
        
        
        #representation of the enviroment: price + open position state
        self._observation_spec = self.construct_obs_spec(self._data_dic, self.propertie_list)

        #used for idndication of the end of episode        
        self._episode_ended = False

        pass
       
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._episode_ended = False
        self._state = self.convert_observation_to_tf(self.env.reset(), self._data_dic, self.propertie_list)
        return ts.restart(self._state)


    def _step(self, action):

        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()
        
        self._state, reward, self._episode_ended, info = self.env.step(action)
        self._state = self.convert_observation_to_tf(self._state, self._data_dic, self.propertie_list)

        if self._episode_ended:
            return ts.termination(self._state, reward)
        else:
            return ts.transition(self._state, reward=reward, discount=1.0)




def convert_obs_to_tf_env_obs(obs):
    cfg = ['ball_owned_team', 
       'steps_left', 
       'ball_owned_player', 
       'game_mode', 
       #'designated', 
       'active', 
       #'left_team_active', 
       'left_team', 
       #'right_team_active', 
       'ball_direction', 
       'ball', 
       #'left_team_tired_factor', 
       'left_team_direction', 
       'score', 
       #'left_team_roles', 
       #'right_team_tired_factor', 
       'right_team', 
       'right_team_direction', 
       #'right_team_yellow_card', 
       #'right_team_roles', 
       'ball_rotation', 
       #'left_team_yellow_card', 
       'sticky_actions']
    
    o = MyFootballEnv.convert_observation_to_tf(obs['players_raw'], data_dic, cfg )
    for key, value in o.items():
        o[key] = tf.expand_dims(tf.convert_to_tensor(value), axis=0)

    return o

policy_dir = './policy/'
saved_policy = tf.compat.v2.saved_model.load(policy_dir)



def agent(obs):
    global saved_policy
    
    st = tf.constant(np.array(np.array([0], dtype=np.int32)))
    rw = tf.constant(np.array(np.array([0], dtype=np.float32)))
    ds = tf.constant(np.array(np.array([0], dtype=np.float32)))
    ts_obs = {}
    ts_obs = convert_obs_to_tf_env_obs(obs)
    t = ts.TimeStep(st, rw, ds, ts_obs)
    
    a = saved_policy.action(t).action.numpy()[0]
    
    return [a]


# Test if all above is working

In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", debug=True, configuration={"save_video": True, "scenario_name": "1_vs_1_easy", 
                                                  "running_in_notebook": True, "episodeSteps": 200})
output = env.run(["do_nothing","submission.py"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)